In [13]:
# Basic package
import pandas as pd
import numpy as np
import datetime as dt

#Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Save model for future use
import pickle

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

np.random.seed(42)

In [2]:
# Display all results in a cell, not just the last line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
df_1 =pd.read_csv('NFL Analytics Challenge Data Set 1.csv')
df_2 =pd.read_csv('NFL Analytics Challenge Data Set 2.csv')
df_3 =pd.read_csv('NFL Analytics Challenge Data Set 3.csv')

In [4]:
# Data cleaning and engineering
# Dataset 1
df_1['date'] =pd.to_datetime(df_1['date']) # Change to datetime format
df_1['away'] =df_1['away'].str.replace('.','').str.strip() # Remove punctuation and whitespace

# Convert text to number then split columns with composite number
df_1_cols_to_split =['third_downs_away','third_downs_home','fourth_downs_away','fourth_downs_home',
                     'comp_att_away','comp_att_home','sacks_away','sacks_home',
                     'penalties_away','penalties_home','redzone_away','redzone_home'
                    ]

dic ={'Jan':'1', 'Feb':'2', 'Mar':'3', 'Apr':'4', 'May':'5', 'Jun':'6',
      'Jul':'7', 'Aug':'8', 'Sep':'9', 'Oct':'10', 'Nov':'11', 'Dec':'12'
     }

for col in df_1_cols_to_split:
    df_1[col] =df_1[col].replace(dic, regex=True)
    df_1 =df_1.join(df_1[col].str.split('-', expand=True).add_prefix(col))
    
# Convert newly created columns to correct datatype to remove leading zeros
for col in df_1.columns[37:]:
    df_1[col] = df_1[col].astype('int64')
    
#df_1.info() #Validate that all columns have correct data type

# Convert two last columns to time in seconds
m =df_1['possession_away'].str.len().max()
df_1['possession_away'] =df_1['possession_away'].str.rjust(m, '0')
df_1['possession_home'] =df_1['possession_home'].str.rjust(m, '0')

df_1['possession_away_in_seconds'] =((df_1['possession_away'].str[:2].astype('int64'))*3600 +
                                            (df_1['possession_away'].str[3:5].astype('int64'))*60 +
                                            (df_1['possession_away'].str[6:].astype('int64')))

df_1['possession_home_in_seconds'] =((df_1['possession_home'].str[:2].astype('int64'))*3600 +
                                            (df_1['possession_home'].str[3:5].astype('int64'))*60 +
                                            (df_1['possession_home'].str[6:].astype('int64')))

In [5]:
# Data cleaning and engineering
# Dataset 2
df_2['date'] =pd.to_datetime(df_2['date']) # Change to datetime format
df_2['away'] =df_2['away'].str.replace('.','').str.strip() # Remove punctuation and whitespace

# Convert text to number then split columns with composite number
df_2_cols_to_split =['third_downs_away','third_downs_home','fourth_downs_away','fourth_downs_home',
                     'comp_att_away','comp_att_home','sacks_away','sacks_home',
                     'penalties_away','penalties_home','redzone_away','redzone_home'
                    ]

for col in df_2_cols_to_split:
    df_2[col] =df_2[col].replace(dic, regex=True)
    df_2 =df_2.join(df_2[col].str.split('-', expand=True).add_prefix(col))
    
# Convert newly created columns to correct datatype to remove leading zeros
for col in df_2.columns[37:]:
    df_2[col] = df_2[col].astype('int64')
    
#df_2.info() #Validate that all columns have correct data type

# Convert two last columns to time in seconds
m =df_2['possession_away'].str.len().max()
df_2['possession_away'] =df_2['possession_away'].str.rjust(m, '0')
df_2['possession_home'] =df_2['possession_home'].str.rjust(m, '0')

df_2['possession_away_in_seconds'] =((df_2['possession_away'].str[:2].astype('int64'))*3600 +
                                            (df_2['possession_away'].str[3:5].astype('int64'))*60 +
                                            (df_2['possession_away'].str[6:].astype('int64')))

df_2['possession_home_in_seconds'] =((df_2['possession_home'].str[:2].astype('int64'))*3600 +
                                            (df_2['possession_home'].str[3:5].astype('int64'))*60 +
                                            (df_2['possession_home'].str[6:].astype('int64')))

In [6]:
# Data cleaning and engineering
# Dataset 3
df_3['date'] =pd.to_datetime(df_3['date']) # Change to datetime format
df_3['away'] =df_3['away'].str.replace('.','').str.strip() # Remove punctuation and whitespace

In [7]:
# Create final dataframe
df = pd.concat([df_1, df_2],ignore_index=True)
df =df.merge(df_3, on=['date','away','home'])

In [8]:
df.info()
df.shape
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5088 entries, 0 to 5087
Data columns (total 65 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   date                        5088 non-null   datetime64[ns]
 1   away                        5088 non-null   object        
 2   home                        5088 non-null   object        
 3   first_downs_away            5088 non-null   int64         
 4   first_downs_home            5088 non-null   int64         
 5   third_downs_away            5088 non-null   object        
 6   third_downs_home            5088 non-null   object        
 7   fourth_downs_away           5088 non-null   object        
 8   fourth_downs_home           5088 non-null   object        
 9   passing_yards_away          5088 non-null   int64         
 10  passing_yards_home          5088 non-null   int64         
 11  rushing_yards_away          5088 non-null   int64       

(5088, 65)

date      away     home  first_downs_away  first_downs_home  \
0 2002-09-05     49ers   Giants                13                21   
1 2002-09-08  Seahawks  Raiders                14                27   
2 2002-09-08      Jets    Bills                18                26   
3 2002-09-08   Vikings    Bears                19                20   
4 2002-09-08  Chargers  Bengals                27                13   

  third_downs_away third_downs_home fourth_downs_away fourth_downs_home  \
0             12-4             16-9               0-0               0-1   
1             11-1             12-7               2-2               1-1   
2              8-2             17-7               0-0               2-2   
3             13-5             13-7               0-0               0-0   
4             10-6             11-4               0-0               0-0   

   passing_yards_away  ...  penalties_home0  penalties_home1  redzone_away0  \
0                 166  ...               10               80              0   
1                 143  ...                5               45              0   
2                 193  ...               10               82              0   
3                 228  ...                4               33              0   
4                 160  ...                9               57              0   

  redzone_away1  redzone_home0 redzone_home1 possession_away_in_seconds  \
0             8              0             6                      99120   
1             2              0             2                      90540   
2             9              0             8                       1266   
3             7              0             6                     113460   
4             7              0             5                     136080   

  possession_home_in_seconds score_away  score_home  
0                     116880         16          13  
1                     125460         17          31  
2                     140880         37          31  
3                     102540         23          27  
4                       1332         34           6  

[5 rows x 65 columns]

In [9]:
# Create classifier column, if home team win, then 1, if home team lose, then 0
df.loc[df['score_away'] < df['score_home'], 'home_win'] = 1 
df.loc[df['score_away'] > df['score_home'], 'home_win'] = 0 

# Convert to percentage
df['comp_att_away_percentage'] =df['comp_att_away0']/df['comp_att_away1']
df['comp_att_home_percentage'] =df['comp_att_home0']/df['comp_att_home1']

# Drop redzone columns
df = df.drop(columns=['redzone_away','redzone_home',
                      'redzone_away0', 'redzone_away1',
                      'redzone_home0','redzone_home1'])

# Create column with SuperBowl winner by year
winner_dict ={2002:'Buccaneers',
             2003:'Patriots',
             2004:'Patriots',
             2005:'Steelers',
             2006:'Colts',
             2007:'Giants',
             2008:'Steelers',
             2009:'Saints',
             2010:'Packers',
             2011:'Giants',
             2012:'Ravens',
             2013:'Seahawks',
             2014:'Patriots',
             2015:'Broncos',
             2016:'Patriots',
             2017:'Eagles',
             2018:'Patriots',
             2019:'Chiefs',
             2020:'Buccaneers',
             2021:'Rams'
             }
df['super_bowl_winner'] =df['date'].dt.year.map(winner_dict)

# create a dictionary that shows teams in conference/division
# AFC vs NFC Conferences
# each has North, South, East, and West Divisions
conf_divisions = {
    'Colts':'AFC South', 
    'Ravens':'AFC North', 
    'Seahawks':'NFC West', 
    'Patriots':'AFC East', 
    'Packers':'NFC North', 
    'Giants':'NFC East',
    'Steelers':'AFC North', 
    'Jets':'AFC East', 
    'Titans':'AFC South', 
    'Panthers':'NFC South', 
    'Eagles':'NFC East', 
    'Chargers':'AFC West',
    'Saints':'NFC South', 
    '49ers':'NFC West', 
    'Falcons':'NFC South', 
    'Texans':'AFC South', 
    'Jaguars':'AFC South', 
    'Chiefs':'AFC West', 
    'Vikings':'NFC North',
    'Broncos':'AFC West', 
    'Cowboys':'NFC East', 
    'Cardinals':'NFC West', 
    'Bengals':'AFC North', 
    'Bills':'AFC East', 
    'Rams':'NFC West', 
    'Lions':'NFC North',
    'Washington':'NFC East', 
    'Raiders':'AFC West', 
    'Browns':'AFC North', 
    'Buccaneers':'NFC South', 
    'Bears':'NFC North', 
    'Dolphins':'AFC East'  
}
df['Conference_Division'] = df['away'].map(conf_divisions)

# Create column Super Bowl Year
df['super_bowl_year'] =df['date'].dt.year

# TO BE DETERMINED how to treat tie record?
#df['away_win'].unique()
#df[~df['away_win'].isin([0,1])] #11 tie record

In [10]:
df

date      away        home  first_downs_away  first_downs_home  \
0    2002-09-05     49ers      Giants                13                21   
1    2002-09-08  Seahawks     Raiders                14                27   
2    2002-09-08      Jets       Bills                18                26   
3    2002-09-08   Vikings       Bears                19                20   
4    2002-09-08  Chargers     Bengals                27                13   
...         ...       ...         ...               ...               ...   
5083 2022-01-23      Rams  Buccaneers                24                20   
5084 2022-01-23     Bills      Chiefs                23                30   
5085 2022-01-30   Bengals      Chiefs                21                24   
5086 2022-01-30     49ers        Rams                16                25   
5087 2022-02-13      Rams     Bengals                18                15   

     third_downs_away third_downs_home fourth_downs_away fourth_downs_home  \
0                12-4             16-9               0-0               0-1   
1                11-1             12-7               2-2               1-1   
2                 8-2             17-7               0-0               2-2   
3                13-5             13-7               0-0               0-0   
4                10-6             11-4               0-0               0-0   
...               ...              ...               ...               ...   
5083             11-4             14-3               0-0               4-2   
5084             14-6             13-8               4-4               1-1   
5085             14-8             12-6               0-0               0-0   
5086              9-3            18-11               0-0               0-1   
5087             15-6             14-3               1-1               3-1   

      passing_yards_away  ...  possession_away_in_seconds  \
0                    166  ...                       99120   
1                    143  ...                       90540   
2                    193  ...                        1266   
3                    228  ...                      113460   
4                    160  ...                      136080   
...                  ...  ...                         ...   
5083                 355  ...                      122880   
5084                 313  ...                       99420   
5085                 243  ...                      129360   
5086                 232  ...                       87660   
5087                 270  ...                      110820   

      possession_home_in_seconds  score_away score_home  home_win  \
0                         116880          16         13       0.0   
1                         125460          17         31       1.0   
2                         140880          37         31       0.0   
3                         102540          23         27       1.0   
4                           1332          34          6       0.0   
...                          ...         ...        ...       ...   
5083                       93120          30         27       0.0   
5084                      131880          36         42       1.0   
5085                      106920          27         24       0.0   
5086                      128340          17         20       1.0   
5087                      105180          23         20       0.0   

     comp_att_away_percentage comp_att_home_percentage super_bowl_winner  \
0                    0.615385                 0.622222        Buccaneers   
1                    0.718750                 0.678571        Buccaneers   
2                    0.800000                 0.666667        Buccaneers   
3                    0.571429                 0.606061        Buccaneers   
4                    0.789474                 0.580645        Buccaneers   
...                       ...                      ...               ...   
5083                 0.736842                 0.555556 

In [11]:
# export to excel
df.to_excel('NFL_Analytics_Challenge_HA_v2.xlsx')

In [12]:
df.dtypes

date                        datetime64[ns]
away                                object
home                                object
first_downs_away                     int64
first_downs_home                     int64
                                 ...      
comp_att_away_percentage           float64
comp_att_home_percentage           float64
super_bowl_winner                   object
Conference_Division                 object
super_bowl_year                      int64
Length: 65, dtype: object

In [28]:
# filter for january and february
df[df['date'].dt.month.isin([1,2])]

date      away        home  first_downs_away  first_downs_home  \
256  2005-01-02   Jaguars     Raiders                13                16   
257  2005-01-02  Steelers       Bills                15                16   
258  2005-01-02   Packers       Bears                17                17   
259  2005-01-02     Lions      Titans                23                15   
260  2005-01-02      Jets        Rams                22                21   
...         ...       ...         ...               ...               ...   
5083 2022-01-23      Rams  Buccaneers                24                20   
5084 2022-01-23     Bills      Chiefs                23                30   
5085 2022-01-30   Bengals      Chiefs                21                24   
5086 2022-01-30     49ers        Rams                16                25   
5087 2022-02-13      Rams     Bengals                18                15   

     third_downs_away third_downs_home fourth_downs_away fourth_downs_home  \
256              18-8             17-5               1-1               4-2   
257              19-8             12-2               1-1               1-1   
258              11-4             15-3               0-1               3-1   
259              15-5             13-5               2-1               0-0   
260              18-5             13-5               1-1               0-0   
...               ...              ...               ...               ...   
5083             11-4             14-3               0-0               4-2   
5084             14-6             13-8               4-4               1-1   
5085             14-8             12-6               0-0               0-0   
5086              9-3            18-11               0-0               0-1   
5087             15-6             14-3               1-1               3-1   

      passing_yards_away  ...  possession_away_in_seconds  \
256                  149  ...                      111180   
257                  105  ...                      126180   
258                  327  ...                      103860   
259                  331  ...                      118680   
260                  144  ...                      148080   
...                  ...  ...                         ...   
5083                 355  ...                      122880   
5084                 313  ...                       99420   
5085                 243  ...                      129360   
5086                 232  ...                       87660   
5087                 270  ...                      110820   

      possession_home_in_seconds  score_away score_home  home_win  \
256                       104820          13          6       0.0   
257                        89820          29         24       0.0   
258                       112140          31         14       0.0   
259                        97320          19         24       1.0   
260                       111000          29         32       1.0   
...                          ...         ...        ...       ...   
5083                       93120          30         27       0.0   
5084                      131880          36         42       1.0   
5085                      106920          27         24       0.0   
5086                      128340          17         20       1.0   
5087                      105180          23         20       0.0   

     comp_att_away_percentage comp_att_home_percentage super_bowl_winner  \
256                  0.535714                 0.384615          Steelers   
257                  2.083333                 0.533333          Steelers   
258                  0.615385                 0.689655          Steelers   
259                  0.673469                 0.545455          Steelers   
260                  0.583333                 0.725000          Steelers   
...                       ...                      ...               ...   
5083                 0.736842                 0.555556 

In [29]:
# update year column by subtracting 1 for games in Jan/Feb
df.loc[df['date'].dt.month.isin([1,2]), 'super_bowl_year'] = df.loc[df['date'].dt.month.isin([1,2]), 'super_bowl_year'] - 1

In [30]:
# check 
df[df['date'].dt.month.isin([1,2])]

date      away        home  first_downs_away  first_downs_home  \
256  2005-01-02   Jaguars     Raiders                13                16   
257  2005-01-02  Steelers       Bills                15                16   
258  2005-01-02   Packers       Bears                17                17   
259  2005-01-02     Lions      Titans                23                15   
260  2005-01-02      Jets        Rams                22                21   
...         ...       ...         ...               ...               ...   
5083 2022-01-23      Rams  Buccaneers                24                20   
5084 2022-01-23     Bills      Chiefs                23                30   
5085 2022-01-30   Bengals      Chiefs                21                24   
5086 2022-01-30     49ers        Rams                16                25   
5087 2022-02-13      Rams     Bengals                18                15   

     third_downs_away third_downs_home fourth_downs_away fourth_downs_home  \
256              18-8             17-5               1-1               4-2   
257              19-8             12-2               1-1               1-1   
258              11-4             15-3               0-1               3-1   
259              15-5             13-5               2-1               0-0   
260              18-5             13-5               1-1               0-0   
...               ...              ...               ...               ...   
5083             11-4             14-3               0-0               4-2   
5084             14-6             13-8               4-4               1-1   
5085             14-8             12-6               0-0               0-0   
5086              9-3            18-11               0-0               0-1   
5087             15-6             14-3               1-1               3-1   

      passing_yards_away  ...  possession_away_in_seconds  \
256                  149  ...                      111180   
257                  105  ...                      126180   
258                  327  ...                      103860   
259                  331  ...                      118680   
260                  144  ...                      148080   
...                  ...  ...                         ...   
5083                 355  ...                      122880   
5084                 313  ...                       99420   
5085                 243  ...                      129360   
5086                 232  ...                       87660   
5087                 270  ...                      110820   

      possession_home_in_seconds  score_away score_home  home_win  \
256                       104820          13          6       0.0   
257                        89820          29         24       0.0   
258                       112140          31         14       0.0   
259                        97320          19         24       1.0   
260                       111000          29         32       1.0   
...                          ...         ...        ...       ...   
5083                       93120          30         27       0.0   
5084                      131880          36         42       1.0   
5085                      106920          27         24       0.0   
5086                      128340          17         20       1.0   
5087                      105180          23         20       0.0   

     comp_att_away_percentage comp_att_home_percentage super_bowl_winner  \
256                  0.535714                 0.384615          Steelers   
257                  2.083333                 0.533333          Steelers   
258                  0.615385                 0.689655          Steelers   
259                  0.673469                 0.545455          Steelers   
260                  0.583333                 0.725000          Steelers   
...                       ...                      ...               ...   
5083                 0.736842                 0.555556 

In [31]:
# update the super bowl winner for rows changed
df['super_bowl_winner'] = df['super_bowl_year'].map(winner_dict)

In [32]:
# check
df

date      away        home  first_downs_away  first_downs_home  \
0    2002-09-05     49ers      Giants                13                21   
1    2002-09-08  Seahawks     Raiders                14                27   
2    2002-09-08      Jets       Bills                18                26   
3    2002-09-08   Vikings       Bears                19                20   
4    2002-09-08  Chargers     Bengals                27                13   
...         ...       ...         ...               ...               ...   
5083 2022-01-23      Rams  Buccaneers                24                20   
5084 2022-01-23     Bills      Chiefs                23                30   
5085 2022-01-30   Bengals      Chiefs                21                24   
5086 2022-01-30     49ers        Rams                16                25   
5087 2022-02-13      Rams     Bengals                18                15   

     third_downs_away third_downs_home fourth_downs_away fourth_downs_home  \
0                12-4             16-9               0-0               0-1   
1                11-1             12-7               2-2               1-1   
2                 8-2             17-7               0-0               2-2   
3                13-5             13-7               0-0               0-0   
4                10-6             11-4               0-0               0-0   
...               ...              ...               ...               ...   
5083             11-4             14-3               0-0               4-2   
5084             14-6             13-8               4-4               1-1   
5085             14-8             12-6               0-0               0-0   
5086              9-3            18-11               0-0               0-1   
5087             15-6             14-3               1-1               3-1   

      passing_yards_away  ...  possession_away_in_seconds  \
0                    166  ...                       99120   
1                    143  ...                       90540   
2                    193  ...                        1266   
3                    228  ...                      113460   
4                    160  ...                      136080   
...                  ...  ...                         ...   
5083                 355  ...                      122880   
5084                 313  ...                       99420   
5085                 243  ...                      129360   
5086                 232  ...                       87660   
5087                 270  ...                      110820   

      possession_home_in_seconds  score_away score_home  home_win  \
0                         116880          16         13       0.0   
1                         125460          17         31       1.0   
2                         140880          37         31       0.0   
3                         102540          23         27       1.0   
4                           1332          34          6       0.0   
...                          ...         ...        ...       ...   
5083                       93120          30         27       0.0   
5084                      131880          36         42       1.0   
5085                      106920          27         24       0.0   
5086                      128340          17         20       1.0   
5087                      105180          23         20       0.0   

     comp_att_away_percentage comp_att_home_percentage super_bowl_winner  \
0                    0.615385                 0.622222        Buccaneers   
1                    0.718750                 0.678571        Buccaneers   
2                    0.800000                 0.666667        Buccaneers   
3                    0.571429                 0.606061        Buccaneers   
4                    0.789474                 0.580645        Buccaneers   
...                       ...                      ...               ...   
5083                 0.736842                 0.555556 

In [ ]:
# delete the conference_division column (what value does it provide?)

In [ ]:
# differentiate between playoffs and regular season???

# correct games played to correct super bowl year
# all games in February and January belong to previous year
# filter games in this range


# identify super bowl winner in each season (DONE/UPDATED)

### TO DO for monday:

In [ ]:
# step 1
# aggregate data by year (sum yards, etc.)
# groupby
# how to group? sum, average

# step 2
# vertically combine all data to get FULL team season statistics

# step 3
# for each year, add column indicating 0 did not win SB, 1 won SB




In [ ]:
# Only considered matches before 2022
df =df[df['date']<'2022-01-01']

# To be discussed
# Only considered no tie matches
df =df[df['home_win'].isin([0,1])]

In [ ]:
X = df.drop(columns=['date', 'away', 'home','third_downs_away', 'third_downs_home',
                     'fourth_downs_away', 'fourth_downs_home', 'comp_att_away', 'comp_att_home',
                     'sacks_away', 'sacks_home', 'penalties_away', 'penalties_home',
                     'redzone_away', 'redzone_home','possession_away','possession_home',
                     'away_win'])

y = df['home_win']